In [1]:
import os  # dealing with directories
import cv2  # working with, mainly resizing, images
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image as im
import random
from sklearn.decomposition import PCA
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

### Uploading the images

<B> I got the uploading images idea from kaggle notebook

In [2]:
train_directory = r'/Users/idobar/Documents/GitHub/FirstFinal/train/'
train_filepath = [train_directory + last_file_name for last_file_name in os.listdir(train_directory)]
train_dogs_filepath = [train_directory + dog_file_name for dog_file_name in os.listdir(train_directory) if 'dog' in dog_file_name]
train_cats_filepath = [train_directory + cat_file_name for cat_file_name in os.listdir(train_directory) if 'cat' in cat_file_name]

In [3]:
def read_image(file_path):
    #grayscaling
    img = cv2.imread(file_path,0)
    #resizing the images to 64,64 in order to get all the images in the same size
    return cv2.resize(img, (64, 64))

In [4]:
def prep_data(images):
    image_len = len(images)
    size = 4096 #64*64
    X = np.ndarray((image_len, size), dtype=np.uint8)
    y = np.zeros((image_len, 1))
    for i, image_file in enumerate(images):
        image = read_image(image_file)
        X[i, :] = np.squeeze(image.reshape((size, 1)))
        if '/dog' in image_file:
            y[i, 0] = 1
        elif '/cat' in image_file:
             y[i, 0] = 0
        
    return X, y

In [5]:
data,target = prep_data(train_filepath)

In [6]:
df = pd.DataFrame(data)
target=pd.DataFrame(target)

In [7]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,19,47,26,58,49,73,70,68,71,73,...,49,53,36,30,34,41,51,54,58,52
1,96,104,114,112,110,111,111,113,116,115,...,117,98,141,117,121,120,113,107,26,49
2,70,157,180,181,189,164,141,146,149,198,...,114,110,112,95,93,87,85,81,68,68
3,26,25,26,28,27,27,35,28,28,27,...,61,22,35,30,39,36,26,52,46,42
4,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255


In [8]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
24995,24,25,28,31,25,28,24,26,40,44,...,76,72,71,75,72,71,80,76,68,78
24996,23,94,94,25,21,62,44,53,42,33,...,165,160,208,184,211,215,93,83,80,110
24997,4,3,3,2,10,6,10,16,23,22,...,170,101,128,133,98,120,118,167,82,101
24998,132,141,137,145,149,154,156,161,168,167,...,1,1,1,1,1,1,1,1,1,1
24999,87,77,76,69,64,58,55,53,60,44,...,170,134,128,189,203,154,100,100,88,52


In [9]:
df.shape

(25000, 4096)

In [13]:
target.head()

,0
0,1.0
1,0.0
2,1.0
3,0.0
4,0.0


<B> After successfully uploading the images lets do some preprocessing 

## Preprocessing 